### Pair Programming ETL Carga II

___


Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [ ]:
import pandas as pd
import mysql.connector